In [ ]:
import pickle
import os

from src.survival_runner import SurvivalResult
from src.rating.base import RatingResult

from tqdm.auto import tqdm

In [ ]:
test_path = "data/rtp_500/split_1_0.5_0.1_0.2_0.2/test.pkl"

In [ ]:
fragments = [
    # "mini_datasets/multisample_mini_0.pkl",
    # "mini_datasets_run1/multisample_mini_0.pkl",
    # "mini_datasets_run2/multisample_mini_0.pkl",
    # "mini_datasets_run3/multisample_mini_0.pkl",
    # "mini_datasets_run7/multisample_mini_0.pkl",
]

In [ ]:
test_data: list[SurvivalResult] = pickle.load(open(test_path, "rb"))
print("Num samples in test data:", len(test_data))


fragment_data: list[list[SurvivalResult]] = []
for fragment in fragments:
    fragment_data.append(pickle.load(open(fragment, "rb")))
    print("Num samples in fragment:", len(fragment_data[-1]))

In [ ]:
test_data[0]

In [ ]:
# now joint all fragments

all_data = [test_data]
all_data.extend(fragment_data)

joint_data: list[SurvivalResult] = []

for i in tqdm(range(len(all_data[0]))):
    survival_results = [fr[i] for fr in all_data]

    new_id = i
    prompt = survival_results[0].prompt

    max_attempts = sum(res.max_attempts for res in survival_results)
    num_attempts = max_attempts
    is_toxic = any(res.is_toxic for res in survival_results)

    # join ratings
    ratings = []
    for res in survival_results:
        ratings.extend(res.ratings)

    new_surv = SurvivalResult(
        id=new_id,
        prompt=prompt,
        max_attempts=max_attempts,
        num_attempts=num_attempts,
        is_toxic=is_toxic,
        ratings=ratings,
    )

    joint_data.append(new_surv)

In [ ]:
joint_data_light = []

for surv in joint_data:

    new_ratings = [RatingResult(text="", scores=rating.scores) for rating in surv.ratings]

    light_surv = SurvivalResult(
        id=surv.id,
        prompt=surv.prompt,
        max_attempts=surv.max_attempts,
        num_attempts=surv.num_attempts,
        is_toxic=surv.is_toxic,
        ratings=new_ratings,
    )

    joint_data_light.append(light_surv)

In [ ]:
test_path_full = "data/test_full.pkl"
test_path_light = "data/test_full_light.pkl"

pickle.dump(joint_data, open(test_path_full, "wb"))
pickle.dump(joint_data_light, open(test_path_light, "wb"))

In [ ]:
# load and test light data
loaded_data_light = pickle.load(open(test_path_light, "rb"))
print("Num samples in loaded light data:", len(loaded_data_light))
for i in range(5):
    print("Sample", i)
    print("Prompt:", loaded_data_light[i].prompt)
    print("Max attempts:", loaded_data_light[i].max_attempts)
    print("Num attempts:", loaded_data_light[i].num_attempts)
    print("Is toxic:", loaded_data_light[i].is_toxic)
    print("Ratings:", loaded_data_light[i].ratings)
    print()

In [ ]:
# load and test 

loaded_data = pickle.load(open(test_path_full, "rb"))
print("Num samples in loaded data:", len(loaded_data))
for i in range(5):
    print("Sample", i)
    print("Prompt:", loaded_data[i].prompt)
    print("Max attempts:", loaded_data[i].max_attempts)
    print("Num attempts:", loaded_data[i].num_attempts)
    print("Is toxic:", loaded_data[i].is_toxic)
    print("Ratings:", loaded_data[i].ratings)
    print()
    
# load and test light data
loaded_data_light = pickle.load(open(test_path_light, "rb"))
print("Num samples in loaded light data:", len(loaded_data_light))
for i in range(5):
    print("Sample", i)
    print("Prompt:", loaded_data_light[i].prompt)
    print("Max attempts:", loaded_data_light[i].max_attempts)
    print("Num attempts:", loaded_data_light[i].num_attempts)
    print("Is toxic:", loaded_data_light[i].is_toxic)
    print("Ratings:", loaded_data_light[i].ratings)
    print()